In [1]:
import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import Point
from geopandas import GeoDataFrame
import os

import seaborn as sns
pd.set_option("display.max_colwidth", 0)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.options.display.float_format = '{:.5f}'.format

import warnings
warnings.filterwarnings("ignore")

In [2]:
%%time
df_localness = pd.read_csv('./data/food_localness_oct.csv',header=0) #,nrows=10
# df_localness

CPU times: user 10.8 s, sys: 4.41 s, total: 15.2 s
Wall time: 21.7 s


In [3]:
df_localness['customer_loc_id'] = df_localness['customer_lon'].astype(str)+df_localness['customer_lat'].astype(str)

## Create weekend binary column

In [4]:
%%time
# Create one column to determine if it is weekend
df_localness['order_date'] = pd.to_datetime(df_localness['order_date']) 

# Define a function to check if a date falls on a weekend
def is_weekend(date):
    return (date.dayofweek == 5) or (date.dayofweek == 6)

# Apply the function to the 'Date' column
df_localness['isweekend'] = df_localness['order_date'].apply(lambda x: is_weekend(x))

CPU times: user 5.13 s, sys: 122 ms, total: 5.25 s
Wall time: 5.52 s


## Merge with mean price per item (proxy for expensiveness of restaurants)

In [5]:
df_priceper_item = pd.read_csv('./data/mean_price_per_item.csv')
df_localness = df_localness.merge(df_priceper_item, how='left',on='vendor_id')
print(df_localness.shape[0])

df_localness = df_localness[df_localness['item_price']!=0]
print(df_localness.shape[0])

2824594
2809076


## merge entropy and restaurant count data

In [8]:
# %%time
# distance_list = [0.5, 1, 2, 3, 4, 5] #unit:km

# # left join various entropy indices and number of resturants
# for distance_threshold in distance_list:
#     entropy_col_name = 'entropy_'+str(distance_threshold)+'km'
#     poi_col_name = 'poi_count_'+str(distance_threshold)+'km'
#     entropy_file_path = './data/entropy_'+str(distance_threshold)+'km_customerlocations_oct.csv'
#     poicount_file_path = './data/poi_count_'+str(distance_threshold)+'km_customerlocations_oct.csv'
    
#     entropy_bycustomerloc1 = pd.read_csv(entropy_file_path,header=0)
#     df_localness = df_localness.merge(entropy_bycustomerloc1, how='left',on='customer_loc_id')
#     df_localness[entropy_col_name] = df_localness[entropy_col_name].fillna(0)
    
#     poi_bycustomerloc1 = pd.read_csv(poicount_file_path,header=0)
#     df_localness = df_localness.merge(poi_bycustomerloc1, how='left',on='customer_loc_id')
#     df_localness[poi_col_name] = df_localness[poi_col_name].fillna(0)
#     print(df_localness.shape[0])

2813376
2813376
2813376
2813376
2813376
2813376
CPU times: user 30.1 s, sys: 20.9 s, total: 51 s
Wall time: 52.8 s


In [6]:
%%time
distance_list = [0.5, 1, 2, 3, 4, 5] #unit:km

# left join various entropy indices and number of resturants
for distance_threshold in distance_list:
    entropy_col_name = 'entropy_'+str(distance_threshold)+'km'
    poi_col_name = 'poi_count_'+str(distance_threshold)+'km'
    entropy_file_path = './data/entropy_'+str(distance_threshold)+'km_customerlocations_oct.csv'
    poicount_file_path = './data/poi_count_'+str(distance_threshold)+'km_customerlocations_oct.csv'
    
    entropy_bycustomerloc1 = pd.read_csv(entropy_file_path,header=0)
    df_localness = df_localness.merge(entropy_bycustomerloc1, how='left',on='order_id')
    df_localness[entropy_col_name] = df_localness[entropy_col_name].fillna(0)
    
    poi_bycustomerloc1 = pd.read_csv(poicount_file_path,header=0)
    df_localness = df_localness.merge(poi_bycustomerloc1, how='left',on='order_id')
    df_localness[poi_col_name] = df_localness[poi_col_name].fillna(0)
    print(df_localness.shape[0])

2809076
2809076
2809076
2809076
2809076
2809076
CPU times: user 24 s, sys: 26.1 s, total: 50.1 s
Wall time: 1min 4s


In [7]:
df_localness['poi_count_0.5_1km'] = df_localness['poi_count_1km'] - df_localness['poi_count_0.5km']
df_localness['poi_count_1_2km'] = df_localness['poi_count_2km'] - df_localness['poi_count_1km']
df_localness['poi_count_2_3km'] = df_localness['poi_count_3km'] - df_localness['poi_count_2km']
df_localness['poi_count_3_4km'] = df_localness['poi_count_4km'] - df_localness['poi_count_3km']
df_localness['poi_count_4_5km'] = df_localness['poi_count_5km'] - df_localness['poi_count_4km']

In [9]:
# df_localness.head(3)

## Merge with rent data

In [8]:
unique_customerlocs_withrent = pd.read_csv('./data/estimated_rent_customerlocations_oct.csv')
print(unique_customerlocs_withrent.shape[0])
unique_customerlocs_withrent.head()

791511


,customer_loc_id,approxi_rent
0,55.21387725.155851,90.44289
1,55.213885121047525.1557155663109,90.65273
2,55.21368825.15563,91.13944
3,55.2138425.155878,90.46375
4,55.213881433010125.1557653365541,90.57859


In [9]:
df_localness = df_localness.merge(unique_customerlocs_withrent, how='left',on='customer_loc_id')
print(df_localness.shape[0])

2809076


In [10]:
df_localness.head(2)

,account_id,order_id,order_date,order_time,vendor_id,vendor_name,main_cuisine,vertical,delivery_fee_amount_lc,service_fee_amount_lc,basket_amount_lc,is_incentive_applied,incentive_amount_lc,pickup_id,customer_lon,customer_lat,vendor_lon,vendor_lat,order_time_hour,delivery_duration,total_duration,node_id,nearest_dist1,poi1,n_vertical,n_vendor_id,n_main_cuisine,n_vendor_name,n_vendor_lon,n_vendor_lat,vendor_index,pandana_distance_realdist,pandana_localness,speed,customer_loc_id,isweekend,item_price,entropy_0.5km,poi_count_0.5km,entropy_1km,poi_count_1km,entropy_2km,poi_count_2km,entropy_3km,poi_count_3km,entropy_4km,poi_count_4km,entropy_5km,poi_count_5km,poi_count_0.5_1km,poi_count_1_2km,poi_count_2_3km,poi_count_3_4km,poi_count_4_5km,approxi_rent
0,1237695,1304769890,2023-10-10,2023-10-10 00:54:08,701885,"Subz, AlBarsha 3",american,food,7.50000,1.92000,64.00000,True,30.00000,249052903.00000,55.14407,25.06988,55.19383,25.08783,0,21,50,2669233716,0.68707,1778,food,660366,american,"Cheat Day - Burgers & Salads, Jumeirah Lakes Towers",55.14022,25.06550,1778,7.29795,0.09415,20.85127,55.14406625.069883,False,NaN,0.60866,70.00000,0.67502,137.00000,0.71957,368.00000,0.71832,376.00000,0.71478,394.00000,0.70749,461.00000,67.00000,231.00000,8.00000,18.00000,67.00000,87.94369
1,1027548,1304769334,2023-10-10,2023-10-10 00:53:16,701885,"Subz, AlBarsha 3",american,food,7.50000,1.92000,64.00000,True,30.00000,249052903.00000,55.14435,25.06983,55.19383,25.08783,0,16,46,2669233716,0.68707,1778,food,660366,american,"Cheat Day - Burgers & Salads, Jumeirah Lakes Towers",55.14022,25.06550,1778,7.29795,0.09415,27.36730,55.1443525.06983,False,NaN,0.50297,37.00000,0.67241,133.00000,0.71957,368.00000,0.71832,376.00000,0.71478,394.00000,0.70749,461.00000,96.00000,235.00000,8.00000,18.00000,67.00000,88.00245


## merge incentives data

In [11]:
df_incentives = pd.read_csv('./data/mit_base_orders_incentives_oct.csv')
df_incentives.head(2)

,order_id,compensation,discounted delivery,free delivery,gem,item discount,menu discount,voucher wallet
0,1293286909,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,0.00000
1,1293286912,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,0.00000


In [12]:
%%time
df_localness = df_localness.merge(df_incentives, on='order_id',how='left')
print(df_localness.shape[0])
df_localness.fillna(0,inplace=True)

2809076
CPU times: user 9.7 s, sys: 4.45 s, total: 14.1 s
Wall time: 14.3 s


In [13]:
# create one categorical var for incentive types
df_localness['incentives_type'] = 'none'
df_localness.loc[df_localness['compensation'] != 0,'incentives_type'] = 'compensation'
df_localness.loc[df_localness['discounted delivery'] != 0,'incentives_type'] = 'discounted delivery'
df_localness.loc[df_localness['free delivery'] != 0,'incentives_type'] = 'free delivery'
df_localness.loc[df_localness['gem'] != 0,'incentives_type'] = 'gem'
df_localness.loc[df_localness['item discount'] != 0,'incentives_type'] = 'item discount'
df_localness.loc[df_localness['menu discount'] != 0,'incentives_type'] = 'menu discount'
df_localness.loc[df_localness['voucher wallet'] != 0,'incentives_type'] = 'voucher wallet'

In [14]:
# create three dummy vars for three most important incentives
df_localness['incentive_delivery'] = 'False'
df_localness.loc[df_localness['discounted delivery'] != 0,'incentive_delivery'] = 'True'
df_localness.loc[df_localness['free delivery'] != 0,'incentive_delivery'] = 'True'

df_localness['incentive_menu_discount'] = 'False'
df_localness.loc[df_localness['menu discount'] != 0,'incentive_menu_discount'] = 'True'

df_localness['incentive_voucher_wallet'] = 'False'
df_localness.loc[df_localness['voucher wallet'] != 0,'incentive_voucher_wallet'] = 'True'

## Merge with rating data

In [15]:
df_ratings = pd.read_csv('./data/restaurant_ratings.csv')
df_ratings = df_ratings[['vendor_id','rating']]
print(df_ratings.shape[0])
print(df_ratings.rating.value_counts())
df_ratings.head()

9349
Very good    6275
Amazing      2239
Good         698 
OK           137 
Name: rating, dtype: int64


,vendor_id,rating
0,728391,Amazing
1,637519,Amazing
2,707758,Amazing
3,673790,Amazing
4,8440,Very good


In [16]:
# left join with the picked resturants
df_localness = df_localness.merge(df_ratings, how='left')
print(df_localness.shape[0])
df_localness['rating'] = df_localness['rating'].fillna('None')
print(df_localness.shape[0])

2809076
2809076


In [17]:
# left join with nearest restaurants
df_ratings = df_ratings.rename(columns={"vendor_id": "n_vendor_id", "rating":"n_rating"})
df_localness = df_localness.merge(df_ratings, how='left')
df_localness['n_rating'] = df_localness['n_rating'].fillna('None')
print(df_localness.shape[0])

2809076


## Merge with individual order consistency indicator: entropy
Lower value of entropy means high consistency

In [18]:
df_consistency = pd.read_csv('./data/order_consistency_oct.csv')
print(df_consistency.shape[0])
df_consistency.head(2)

563561


,account_id,consistency
0,1008,1.00000
1,4626,0.76372


In [19]:
df_localness = df_localness.merge(df_consistency, how='left')
print(df_localness['consistency'].isnull().values.any())
print(df_localness.shape[0])

False
2809076


## Merge with total number of orders in one month for each individual
entropy of consistency itself may be biased so also incorporate total number of orders each person placed in Oct

In [20]:
df_totalorders = df_localness[['account_id','vendor_id']].groupby('account_id').count().reset_index()
df_totalorders.columns = ['account_id', 'total_orders']
# df_totalorders.head()
df_localness = df_localness.merge(df_totalorders, how='left')
print(df_localness['total_orders'].isnull().values.any())
print(df_localness.shape[0])

False
2809076


## Rename columns to avoid confusions

In [21]:
# exclude rows without rent data
print('no rent data points:',df_localness[df_localness.approxi_rent==0].shape[0])
df_localness_filter = df_localness[df_localness.approxi_rent!=0]
print(df_localness_filter.shape[0])

no rent data points: 30354
2778722


In [22]:
df_localness_filter.loc[df_localness_filter.rating == 'Amazing', 'rating'] = '1.Amazing'
df_localness_filter.loc[df_localness_filter.rating == 'Very good', 'rating'] = '2.Very good'
df_localness_filter.loc[df_localness_filter.rating == 'Good', 'rating'] = '3.Good'
df_localness_filter.loc[df_localness_filter.rating == 'OK', 'rating'] = '4.OK'
df_localness_filter.loc[df_localness_filter.rating == 'None', 'rating'] = '5.None'

df_localness_filter.loc[df_localness_filter.n_rating == 'Amazing', 'n_rating'] = '1.Amazing'
df_localness_filter.loc[df_localness_filter.n_rating == 'Very good', 'n_rating'] = '2.Very good'
df_localness_filter.loc[df_localness_filter.n_rating == 'Good', 'n_rating'] = '3.Good'
df_localness_filter.loc[df_localness_filter.n_rating == 'OK', 'n_rating'] = '4.OK'
df_localness_filter.loc[df_localness_filter.n_rating == 'None', 'n_rating'] = '5.None'

df_localness_filter.loc[df_localness_filter.isweekend == True, 'isweekend'] = 'Yes'
df_localness_filter.loc[df_localness_filter.isweekend == False, 'isweekend'] = 'No'

df_localness_filter.loc[df_localness_filter.incentive_delivery == True, 'incentive_delivery'] = 'Yes'
df_localness_filter.loc[df_localness_filter.incentive_delivery == False, 'incentive_delivery'] = 'No'

df_localness_filter.loc[df_localness_filter.incentive_menu_discount == True, 'incentive_menu_discount'] = 'Yes'
df_localness_filter.loc[df_localness_filter.incentive_menu_discount == False, 'incentive_menu_discount'] = 'No'

df_localness_filter.loc[df_localness_filter.incentive_voucher_wallet == True, 'incentive_voucher_wallet'] = 'Yes'
df_localness_filter.loc[df_localness_filter.incentive_voucher_wallet == False, 'incentive_voucher_wallet'] = 'No'

In [23]:
df_localness_filter = df_localness_filter.rename(columns={"item_price": "price_per_item", "approxi_rent":"estimated_rent",
  "poi_count_0.5km" : "restaurant_count_0.5km", "poi_count_0.5_1km": "restaurant_count_0.5_1km",
  "poi_count_1_2km": "restaurant_count_1_2km", "poi_count_2_3km" : "restaurant_count_2_3km", 
  "poi_count_3_4km" : "restaurant_count_3_4km", "poi_count_4_5km" : "restaurant_count_4_5km"
})

## Correct localness value

In [24]:
df_localness_filter.head(2)

,account_id,order_id,order_date,order_time,vendor_id,vendor_name,main_cuisine,vertical,delivery_fee_amount_lc,service_fee_amount_lc,basket_amount_lc,is_incentive_applied,incentive_amount_lc,pickup_id,customer_lon,customer_lat,vendor_lon,vendor_lat,order_time_hour,delivery_duration,total_duration,node_id,nearest_dist1,poi1,n_vertical,n_vendor_id,n_main_cuisine,n_vendor_name,n_vendor_lon,n_vendor_lat,vendor_index,pandana_distance_realdist,pandana_localness,speed,customer_loc_id,isweekend,price_per_item,entropy_0.5km,restaurant_count_0.5km,entropy_1km,poi_count_1km,entropy_2km,poi_count_2km,entropy_3km,poi_count_3km,entropy_4km,poi_count_4km,entropy_5km,poi_count_5km,restaurant_count_0.5_1km,restaurant_count_1_2km,restaurant_count_2_3km,restaurant_count_3_4km,restaurant_count_4_5km,estimated_rent,compensation,discounted delivery,free delivery,gem,item discount,menu discount,voucher wallet,incentives_type,incentive_delivery,incentive_menu_discount,incentive_voucher_wallet,rating,n_rating,consistency,total_orders
0,1237695,1304769890,2023-10-10,2023-10-10 00:54:08,701885,"Subz, AlBarsha 3",american,food,7.50000,1.92000,64.00000,True,30.00000,249052903.00000,55.14407,25.06988,55.19383,25.08783,0,21,50,2669233716,0.68707,1778,food,660366,american,"Cheat Day - Burgers & Salads, Jumeirah Lakes Towers",55.14022,25.06550,1778,7.29795,0.09415,20.85127,55.14406625.069883,No,0.00000,0.60866,70.00000,0.67502,137.00000,0.71957,368.00000,0.71832,376.00000,0.71478,394.00000,0.70749,461.00000,67.00000,231.00000,8.00000,18.00000,67.00000,87.94369,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,0.00000,menu discount,False,True,False,5.None,2.Very good,0.87874,11
1,1027548,1304769334,2023-10-10,2023-10-10 00:53:16,701885,"Subz, AlBarsha 3",american,food,7.50000,1.92000,64.00000,True,30.00000,249052903.00000,55.14435,25.06983,55.19383,25.08783,0,16,46,2669233716,0.68707,1778,food,660366,american,"Cheat Day - Burgers & Salads, Jumeirah Lakes Towers",55.14022,25.06550,1778,7.29795,0.09415,27.36730,55.1443525.06983,No,0.00000,0.50297,37.00000,0.67241,133.00000,0.71957,368.00000,0.71832,376.00000,0.71478,394.00000,0.70749,461.00000,96.00000,235.00000,8.00000,18.00000,67.00000,88.00245,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,0.00000,menu discount,False,True,False,5.None,2.Very good,0.88989,23


In [25]:
df_localness_filter['nearest_dist1'] = df_localness_filter['nearest_dist1']+0.001 # plus one meter to correct zero localness when both pandana_distance_realdist and nearest_dist1 are zero
df_localness_filter['pandana_distance_realdist'] = df_localness_filter['pandana_distance_realdist']+0.001
df_localness_filter['pandana_localness'] = df_localness_filter['nearest_dist1']/df_localness_filter['pandana_distance_realdist']

In [28]:
print(df_localness_filter.shape[0])

2778722


## Save processed data for ML model

In [27]:
%%time
df_localness_filter.to_csv('./data/food_localness_final_oct_new.csv',index=False)

CPU times: user 1min 50s, sys: 1.6 s, total: 1min 51s
Wall time: 1min 54s


In [52]:
# %%time
# df_localness.to_csv('./data/food_localness_final_oct(withoutfilter_rent).csv',index=False)

CPU times: user 2min 38s, sys: 1.96 s, total: 2min 40s
Wall time: 4min 49s
